In [1]:
import requests
from bs4 import BeautifulSoup
res = requests.get("https://ifoodie.tw/")
soup = BeautifulSoup(res.text,'html')
next_url = soup.find_all("div",class_="jsx-1638812939 category-block-item-outer")[0].a["href"]
res = requests.get(f'https://ifoodie.tw{next_url}')
soup = BeautifulSoup(res.text,'html')
tag = soup.find("div",class_="jsx-1156793088 info-rows")
store = tag.a.text
time = tag.div
print(time)

<div class="jsx-1156793088 title-row"><div class="jsx-1156793088 title"><span class="jsx-1156793088 index">1.</span><a class="jsx-1156793088 title-text" href="/restaurant/559d1fdbc03a103ee86c41ee-%E9%80%99%E4%B8%80%E9%8D%8B" target="_self">這一鍋</a></div><svg aria-hidden="true" class="jss41 btn-save" focusable="false" role="presentation" title="儲存餐廳至個人收藏" viewbox="0 0 24 24"><path d="M17 3H7c-1.1 0-1.99.9-1.99 2L5 21l7-3 7 3V5c0-1.1-.9-2-2-2zm0 15l-5-2.18L7 18V5h10v13z"></path><path d="M0 0h24v24H0z" fill="none"></path></svg></div>


In [2]:
import requests
from bs4 import BeautifulSoup
import json

def parse_html(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = []

    restaurant_items = soup.find_all('div', class_='restaurant-item')
    for item in restaurant_items:
        title = item.find('a', class_='title-text').text.strip()
        
        rating_div = item.find('div', class_='rating-star')
        rating = rating_div.find('div', class_='text').text.strip() if rating_div else None

        extra_info = item.find('div', class_='jsx-1156793088 info')
        if extra_info:
            time_text = extra_info.text.strip().replace("今日營業: ", "")
            time_data = [time.strip() for time in time_text.split(',')]
        else:
            time_data = []

        avg_price_elem = item.find('div', class_='avg-price')
        avg_price = avg_price_elem.text.strip().replace('均消 ', '') if avg_price_elem else None

        address_elem = item.find('div', class_='address-row')
        address = address_elem.text.strip() if address_elem else None

        categories = item.find('div', class_='category-row').find_all('a', class_='category')
        categories = [category.text.strip() for category in categories]

        article = {
            '店名': title,
            '評分': rating,
            '營業時間': time_data,
            '金額': avg_price,
            '地點': address,
            '品項': categories
        }

        articles.append(article)
        

    return articles


def get_all_pages_data(base_url):
    all_articles = []

    page = 1
    while True:
        url = f"{base_url}?page={page}"
        try:
            articles = parse_html(url)
            if not articles:  # 如果沒有資料則跳出迴圈
                break
            all_articles.extend(articles)
            page += 1
            print(page)
        except AttributeError as e:
            print(f"AttributeError on page {page}: {str(e)}")
            break

    return all_articles

base_url = 'https://ifoodie.tw/explore/%E5%8F%B0%E5%8C%97%E5%B8%82/list/%E5%B1%85%E9%85%92%E5%B1%8B'
articles = get_all_pages_data(base_url)
json_data = json.dumps(articles, ensure_ascii=False, indent=4)
print(json_data)



2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
[
    {
        "店名": "品都串燒攤",
        "評分": "4.5",
        "營業時間": [
            "17:30-00:00"
        ],
        "金額": "· $250",
        "地點": "臺北市大安區延吉街83號",
        "品項": [
            "附近餐廳",
            "日本料理",
            "居酒屋",
            "宵夜",
            "晚餐"
        ]
    },
    {
        "店名": "雞老闆 桶仔雞 萬隆店",
        "評分": "4.4",
        "營業時間": [
            "00:00-02:30",
            "17:00-00:00"
        ],
        "金額": "· $600",
        "地點": "臺北市文山區羅斯福路五段69號",
        "品項": [
            "附近餐廳",
            "日本料理",
            "居酒屋",
            "宵夜",
            "晚餐"
        ]
    },
    {
        "店名": "起家莊雞老闆 養生藥膳桶仔雞",
        "評分": "4.5",
        "營業時間": [
            "00:00-02:30",
            "17:00-00:00"
        ],
        "金額": "· $500",
        "地點": "臺北市松山區長春路454號",
        "品項": [
            "附近餐廳",
            "日本料理",
            "燒烤",
            "居酒屋",
            "宵夜"
     

In [3]:
import json
json_data=json.loads(json_data)
for item in json_data:
    shop_name = item["店名"]
    business_hours = ", ".join(item["營業時間"])
    price = item["金額"]
    location = item["地點"]
    categories = ", ".join(item["品項"])

    # 在这里可以对提取到的数据进行进一步处理或打印输出
    print("店名:", shop_name)
    print("營業時間:", business_hours)
    print("金額:", price)
    print("地點:", location)
    print("品項:", categories)
    print()

店名: 品都串燒攤
營業時間: 17:30-00:00
金額: · $250
地點: 臺北市大安區延吉街83號
品項: 附近餐廳, 日本料理, 居酒屋, 宵夜, 晚餐

店名: 雞老闆 桶仔雞 萬隆店
營業時間: 00:00-02:30, 17:00-00:00
金額: · $600
地點: 臺北市文山區羅斯福路五段69號
品項: 附近餐廳, 日本料理, 居酒屋, 宵夜, 晚餐

店名: 起家莊雞老闆 養生藥膳桶仔雞
營業時間: 00:00-02:30, 17:00-00:00
金額: · $500
地點: 臺北市松山區長春路454號
品項: 附近餐廳, 日本料理, 燒烤, 居酒屋, 宵夜

店名: 古記雞 信義店 私房菜居酒屋
營業時間: 15:30-23:30
金額: · $800
地點: 臺北市信義區忠孝東路五段236巷3弄13號
品項: 附近餐廳, 日本料理, 精緻高級, 約會餐廳, 居酒屋

店名: 揪餖燒肉
營業時間: 11:00-23:00
金額: · $700
地點: 臺北市信義區松壽路12號6樓
品項: 附近餐廳, 燒烤, 日本料理, 居酒屋, 精緻高級

店名: 樵 Kikori Yakitori Bar
營業時間: 18:00-23:30
金額: · $1600
地點: 臺北市大安區東豐街38號
品項: 附近餐廳, 日本料理, 精緻高級, 約會餐廳, 居酒屋

店名: 豚馬日本料理
營業時間: 11:30-14:00, 17:30-21:00
金額: · $500
地點: 臺北市士林區福華路152號
品項: 附近餐廳, 日本料理, 居酒屋, 精緻高級, 約會餐廳

店名: 竹崎居酒食堂
營業時間: 18:00-00:00
金額: · $600
地點: 臺北市大同區鄭州路3之2號
品項: 附近餐廳, 日本料理, 約會餐廳, 居酒屋, 宵夜

店名: 肴合清酒吧
營業時間: 17:00-00:00
金額: · $150 - $600
地點: 臺北市信義區中坡南路22巷7號1F
品項: 附近餐廳, 酒吧, 日本料理, 精緻高級, 居酒屋

店名: 暖男炸雞
營業時間: 11:30-23:30
金額: · $300
地點: 臺北市信義區永吉路30巷122號旁的空地
品項: 附近餐廳, 日本料理, 小吃, 居酒屋, 午餐

店名: 大河屋 燒肉丼 串燒 

In [4]:
import pandas as pd

# ... 其他程式碼 ...

# 將資料轉換為 DataFrame
df = pd.DataFrame(json_data)

# 建立新的 Excel 檔案
file_path = "居酒屋.xlsx"
sheet_name = "Sheet1"

# 將 DataFrame 寫入 Excel 檔案
df.to_excel(file_path, sheet_name=sheet_name, index=False)

print("資料成功匯入 Excel 檔案！")

資料成功匯入 Excel 檔案！
